# 10 - Modules and Packages

## Making a Hierarchical Package of Modules
Making a package structure is simple. Just organize your code as you wish on the filesystem and make sure that every directory defines an __init__.py file.

## Controlling the Import of Everything
You want precise control when the user goes import *.

Define a variable __all__ in your module that explicitly lists the exported names.

In [1]:
import os

code = '''
def spam():
    pass

def grok():
    pass

blah = 42

# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']

'''

file_name = "somemodule.py"
with open(file_name, "w") as target_file:
    target_file.write(code)


In [3]:
%%bash

cat somemodule.py


def spam():
    pass

def grok():
    pass

blah = 42

# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']



In [10]:
from somemodule import *

"spam" in globals() and "grok" in globals()

True

In [11]:
"blah" in globals()

False

In [12]:
import somemodule as sm

"blah" in vars(sm)

True

So by using __all__ we prevent blah from entering globals.

## Importing Package Submodules Using Relative Names

In [ ]:
# mypackage/A/spam.py

from mypackage.A import grok # OK
from . import grok           # OK
import grok                  # Error (not found)


The downside of using an absolute name, such as mypackage.A, is that it hardcodes the top-level package name into your source code. If you ever changed the name of the package, you would have to go through all of your files and fix the source code.

The . and .. syntax on the import statement might look funny, but think of it as specifying a directory name. . means look in the current directory and ..B means look in the ../B directory.

## Splitting a Module into Multiple Files
A program module can be split into separate files by turning it into a package.

## Making Separate Directories of Code Import Under a Common Namespace

Instead of having each part installed as a separated named package, you would like all of the parts to join together under a common package prefix.

You would like to define a top-level Python package that serves as a namespace for a large collection of separately maintained subpackages.


In [15]:
setup = '''
foo-package/
    spam/
        blah.py
 
bar-package/
    spam/
        grok.py
'''